# Example01

- 基本的に，[このチュートリアル](https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html)の内容に基づいている．  

## NumPy

NumPyによるFully connected neural networkの実装例

- 3 layers = 隠し層は１つ
- 活性化関数はランプ(ReLU)

In [1]:
# -*- coding: utf-8 -*-
import numpy as np

# N: バッチサイズ ; D_in: 入力の次元;
# H: 隠し層の数   ; D_out: 出力の次元.
N, D_in, H, D_out = 64, 1000, 100, 10

# 乱数で入力と出力データを生成
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# ネットワークの重みも乱数で初期化
w1_init = np.random.randn(D_in, H)
w2_init = np.random.randn(H   , D_out)

w1 = w1_init.copy()
w2 = w2_init.copy()

learning_rate = 1e-6
for t in range(500):
    # 正方向の計算でyを予測
    h = x.dot(w1)
    h_relu = np.maximum(h, 0) ## Ramp function
    y_pred = h_relu.dot(w2)

    # lossの計算
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # lossに従い，w1とw2を計算するための逆伝搬
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # アップデート
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 598.7873717957709
199 2.2070064532551257
299 0.01403328830393357
399 0.00011317743710444353
499 1.0191456599808915e-06


## PyTorch

- NumPyでの実装と同様な振る舞いをPyTorchで再現  
- ソースではPyTorchも乱数初期化しているが，なるべく似た同様な結果が出るように  
  初期値もNumPyから変換した値を使うように変えた．


In [2]:
# -*- coding: utf-8 -*-
import torch

dtype = torch.float
device = torch.device("cpu")
#device = torch.device("cuda:0") # Uncomment this to run on GPU

# 乱数で入力と出力データを生成
x = torch.tensor(x, device=device, dtype=dtype)
y = torch.tensor(y, device=device, dtype=dtype)

# ネットワークの重みも乱数で初期化
w1 = torch.tensor(w1_init, device=device, dtype=dtype)
w2 = torch.tensor(w2_init, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # 正方向の計算でyを予測
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # lossの計算
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # lossに従い，w1とw2を計算するための逆伝搬
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # アップデート
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

99 598.7894287109375
199 2.2069709300994873
299 0.014140358194708824
399 0.00030241423519328237
499 4.687643377110362e-05


## Autograd